In [1]:
import pandas as pd

#pd.set_option("display.max_rows", 100)
#pd.set_option("display.max_columns", 100)

In [2]:
df = pd.read_parquet("muestreos_parcelas.parquet")

estados = [
    "estado_fenologico_1",
    "estado_fenologico_2",
    "estado_fenologico_3",
    "estado_fenologico_4",
    "estado_fenologico_5",
    "estado_fenologico_6",
    "estado_fenologico_7",
    "estado_fenologico_8",
    "estado_fenologico_9",
    "estado_fenologico_10",
    "estado_fenologico_11",
    "estado_fenologico_12",
    "estado_fenologico_13",
    "estado_fenologico_14",
]

In [3]:
def find_estado_with_value_two(row) -> int:
    """
    Returns numero del siguiente etado (yt+1). (e.g., 13)
    """
    for column in row.index:
        if row[column] == 2:
            number_growth_stage = int(column.split("_")[-1])
            return number_growth_stage


def get_valid_dataset(df: pd.DataFrame(), max_days_till_next_date: int) -> pd.DataFrame():
    
    # Removing null and filling empty estados with 0
    df = df.dropna(subset=estados, how="all")
    df = df.dropna(subset=["codparcela"])
    df[estados] = df[estados].fillna(0)

    #Filtering Dataset to keep rows only with one unique 2
    df["count_2s"] = df[estados].eq(2).sum(axis=1)
    df = df[df.count_2s == 1]

    # Sorting by date
    df["fecha"] = pd.to_datetime(df["fecha"])
    df.sort_values(by="fecha", inplace=True)

    # Creating a column to display the number of days till the next observation
    df["next_date"] = df.groupby("codparcela", observed=True)["fecha"].shift(-1)
    df["days_until_next_visit"] = (df["next_date"] - df["fecha"]).dt.days

    # Creating a column to display the next estado_fenológico (yt+1)
    df["estado_actual"] = df[estados].apply(find_estado_with_value_two, axis=1)
    df["next_estado"] = df.groupby("codparcela", observed=True)["estado_actual"].shift(-1)

    # Removing the parcels with only one entry and the last entry for every parcel
    df = df.dropna(subset=["days_until_next_visit"])  # 5150 entries removed

    #Removing entries where estado decreases
    mask_estado_decrease = df["next_estado"] - df["estado_actual"] < 0
    df = df[~mask_estado_decrease]

    #Changing datatypes
    df["days_until_next_visit"] = df["days_until_next_visit"].astype("int16")
    df["next_estado"] = df["next_estado"].astype("int8")
    df[estados] = df[estados].astype("int8")

    # Filtering the max days
    df = df[df["days_until_next_visit"] < max_days_till_next_date]

    excluded_columns = ['estado_actual','count_2s', 'next_date']

    return df.loc[:, [i for i in df.columns if i not in excluded_columns]]


In [4]:
df_new = get_valid_dataset(df, 10)

In [5]:
df_new

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,...,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,days_until_next_visit,next_estado
388249,2020-04-23 17:00:15,001-00163-00-00,malaga,archidona,2005-08-30,2005,1,163,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,10
374542,2020-04-23 17:00:15,001-00162-00-00,malaga,antequera,2005-08-30,2005,1,162,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,10
386907,2020-04-23 17:00:15,166-00500-00-00,malaga,antequera,2005-08-31,2005,166,500,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,10
375863,2020-04-23 17:00:15,011-00001-00-00,malaga,antequera,2005-08-31,2005,11,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,10
385226,2020-04-23 17:00:15,110-00004-00-00,malaga,antequera,2005-08-31,2005,110,4,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558747,2021-04-05 17:26:06,004-00069-02-05,sevilla,marinaleda,2021-03-24,2021,4,69,2,5,...,DOCTORA,NaN,NaN,NaN,NaN,NaN,SI,Producción Integrada (PI),7,4
558617,2021-04-05 17:26:06,002-00087-08-01,sevilla,marinaleda,2021-03-24,2021,2,87,8,1,...,MOSTAZARES,NaN,NaN,NaN,NaN,NaN,SI,Producción Integrada (PI),7,4
554133,2021-04-05 17:26:06,012-00030-01-1,sevilla,gilena,2021-03-24,2021,12,30,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Producción Integrada (PI),5,3
570933,2021-04-05 17:26:06,009-00033-02-02,sevilla,pedrera,2021-03-24,2021,9,33,2,2,...,MINERIA CERRO DEL OJO,NaN,NaN,NaN,NaN,NaN,SI,Producción Integrada (PI),7,3


## EDA Diferencia Coordenadas para una misma parcela

In [6]:
df_new[df_new["102_coordenada_x_(utm)"].isnull()].codparcela.value_counts()

codparcela
001-00048-00-50    171
019-00160-00-10    109
014-00018-00-00    106
075-00019-00-00    106
005-00044-00-00    101
                  ... 
009-00107-01-01      0
009-00106-03-01      0
009-00106-02-00      0
009-00106-00-00      0
015-00013-01         0
Name: count, Length: 5239, dtype: int64

In [7]:
df_new[df_new['codparcela']=='001-00048-00-50']["102_coordenada_x_(utm)"].value_counts()

102_coordenada_x_(utm)
435359.0    40
435109.0    19
Name: count, dtype: int64

In [8]:
df_new[df_new['codparcela']=='001-00048-00-50']["103_coordenada_y_(utm)"].value_counts()

103_coordenada_y_(utm)
4126802.0     40
41326610.0    19
Name: count, dtype: int64

In [9]:
grouped = df_new.groupby('codparcela', observed=True)["102_coordenada_x_(utm)"]
df_new['max_UTC'] = grouped.transform('max')
df_new['min_UTC'] = grouped.transform('min')
df_new['difference'] = df_new['max_UTC'] - df_new['min_UTC']

In [10]:
df_diff = df_new['difference'].value_counts()
df_diff[df_diff.index>10000].sum() / len(df)

0.07452306920158888

In 8% of the dataset the parcels have a difference of more than 10km between observations in their x_coordinate.

In [11]:
grouped = df_new.groupby('codparcela', observed=True)["103_coordenada_y_(utm)"]
df_new['max_UTC_y'] = grouped.transform('max')
df_new['min_UTC_y'] = grouped.transform('min')
df_new['difference_y'] = df_new['max_UTC_y'] - df_new['min_UTC_y']

In [12]:
df_diff = df_new['difference_y'].value_counts()
df_diff[df_diff.index>10000].sum() / len(df)

0.07906935972072542

In 8.5% of the dataset the parcels have a difference of more than 10km between observations in their y_coordinate.